
# CRIM Intervals:  Melodic and Harmonic Corpus Search

### What You Can Do with this Notebook:

* Search A Corpus for Melodic and Harmonic nGrams

### A. Import Intervals and Other Code


In [1]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os
import glob as glob


MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)
else:
    print(MYDIR, "folder already exists.")
    
MUSDIR = ("Music_Files")
CHECK_FOLDER = os.path.isdir(MUSDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MUSDIR)
    print("created folder : ", MUSDIR)
else:
    print(MUSDIR, "folder already exists.")

saved_csv folder already exists.
Music_Files folder already exists.


## B. Corpus Inventory

The **CorpusBase** class is a convenient way to find patterns in any given list of pieces.
    
See the **Corpus Methods** Notebook for details, and `print(CorpusBase.batch.__doc__)`

### B.1  Corpus Melodic Inventory

- Also see Corpus Methods Notebook for other ways to import local and remote files!

- NB: use `ImportedPiece`, not `piece`!
- NB:  `func1` and `func2` do **NOT** include the closing parentheses!

```
corpus = CorpusBase(['https://crimproject.org/mei/CRIM_Mass_0050_1.mei',
                     'https://crimproject.org/mei/CRIM_Mass_0050_2.mei'
                     ])
func1 = ImportedPiece.melodic
list_of_dfs = corpus.batch(func=func1, kwargs={'kind': 'd', 'end': False}, metadata=False)
func2 = ImportedPiece.ngrams
list_of_melodic_ngrams = corpus.batch(func=func2, kwargs={'n': 4, 'df': list_of_dfs}, metadata=True)
title_of_output = pd.concat(list_of_melodic_ngrams)



#### Import Corpus with URLs

In [4]:
#  first the list of pieces
corpus = CorpusBase(['http://dev.crimproject.org/mei/CRIM_Model_0016.mei',
                     'http://dev.crimproject.org/mei/CRIM_Model_0017.mei'
                     ])

### Corpus Results for Melodic Ngrams

In [5]:

func1 = ImportedPiece.melodic
list_of_dfs = corpus.batch(func=func1, kwargs={'kind': 'd', 'end': False}, metadata=False)
func2 = ImportedPiece.ngrams
list_of_melodic_ngrams = corpus.batch(func=func2, kwargs={'n': 5, 'df': list_of_dfs}, metadata=False)
func3 = ImportedPiece.detailIndex
list_of_detail_index = corpus.batch(func=func3, kwargs={'offset': False,'df': list_of_melodic_ngrams}, metadata=True)

mel_corpus = pd.concat(list_of_detail_index)
comp = mel_corpus.pop("Composer")
mel_corpus['Composer'] = comp
title = mel_corpus.pop("Title")
mel_corpus["Title"] = title
mel_corpus = mel_corpus.fillna('-')
mel_corpus

1                   2                  3                  4  5  \
Measure Beat                                                                   
1       1.0   -    (1, -2, 2, 1, 2)                  -                  -  -   
2       1.0   -    (-2, 2, 1, 2, 2)                  -                  -  -   
        3.0   -    (2, 1, 2, 2, -2)                  -                  -  -   
3       1.0   -   (1, 2, 2, -2, -2)                  -   (1, -2, 2, 1, 2)  -   
4       1.0   -  (2, 2, -2, -2, -2)                  -   (-2, 2, 1, 2, 2)  -   
...          ..                 ...                ...                ... ..   
171     4.0   -                   -   (-5, 2, 2, 2, 2)   (2, 2, 2, 2, -5)  -   
        4.5   -                   -                  -   (2, 2, 2, -5, 8)  -   
172     1.0   -                   -   (2, 2, 2, 2, -5)  (2, 2, -5, 8, -3)  -   
        1.5   -                   -   (2, 2, 2, -5, 5)                  -  -   
        2.0   -                   -  (2, 2, -5, 5, -2)                  -  -   

                              6          Composer  \
Measure Beat                                        
1       1.0                   -  Josquin Des Prés   
2       1.0                   -  Josquin Des Prés   
        3.0                   -  Josquin Des Prés   
3       1.0                   -  Josquin Des Prés   
4       1.0                   -  Josquin Des Prés   
...                         ...               ...   
171     4.0                   -  Josquin Des Prés   
        4.5   (2, -5, 3, -3, 1)  Josquin Des Prés   
172     1.0   (-5, 3, -3, 1, 4)  Josquin Des Prés   
        1.5                   -  Josquin Des Prés   
        2.0                   -  Josquin Des Prés   

                                       Title  
Measure Beat                                  
1       1.0   Mente Tota (from Vultuum tuum)  
2       1.0   Mente Tota (from Vultuum tuum)  
        3.0   Mente Tota (from Vultuum tuum)  
3       1.0   Mente Tota (from Vultuum tuum)  
4       1.0   Mente Tota (from Vultuum tuum)  
...                                      ...  
171     4.0                     Benedicta es  
        4.5                     Benedicta es  
172     1.0                     Benedicta es  
        1.5                     Benedicta es  
        2.0                     Benedicta es  

[883 rows x 8 columns]

In [77]:
piece_list = ['https://crimproject.org/mei/CRIM_Model_0008.mei', 
              'https://crimproject.org/mei/CRIM_Mass_0001_2.mei']
list_ng_dfs = []

for piece in piece_list:
    piece = importScore(piece)
    title = piece.metadata['title']
    mel = piece.melodic(kind="d")
    ng = piece.ngrams(df=mel, n=4).stack().value_counts().reset_index()
    ng.rename(columns = {'index':'ngram', 0: 'counts'}, inplace=True)
    ng['Title'] = title
    list_ng_dfs.append(ng)
all_ngs = pd.concat([list_ng_dfs[0], list_ng_dfs[1]])
all_ngs.sort_values(by=['ngram'], inplace=True)


,ngram,counts,Title
0,"(-2, -2, -2, -2)",34,Ave Maria
5,"(-2, -2, -2, -2)",10,Missa Confitemini: Gloria
51,"(-2, -2, -2, -3)",4,Ave Maria
259,"(-2, -2, -2, 1)",1,Missa Confitemini: Gloria
14,"(-2, -2, -2, 1)",8,Ave Maria
...,...,...,...
126,"(8, -3, 1, 4)",1,Ave Maria
147,"(8, -4, 2, -2)",1,Ave Maria
215,"(8, 1, 1, -5)",1,Ave Maria
302,"(8, 1, 1, 1)",1,Missa Confitemini: Gloria


In [105]:
piece_list = ['https://crimproject.org/mei/CRIM_Model_0008.mei', 
              'https://crimproject.org/mei/CRIM_Mass_0001_2.mei']
list_ng_dfs = []

for piece in piece_list:
    piece = importScore(piece)
    title = piece.metadata['title']
    mel = piece.melodic(kind="d")
    ng = piece.ngrams(df=mel, n=4).stack().reset_index()
    ng.rename(columns = {0:'ngram', 'level_0': 'offset', 'level_1': 'voice'}, inplace=True)
    ng['Title'] = title

    list_ng_dfs.append(ng)
all_ngs = pd.concat([list_ng_dfs[0], list_ng_dfs[1]])
all_ngs = all_ngs.groupby(by=["ngram", "Title"]).size().to_frame().reset_index()
all_ngs.rename(columns = {0:'count'})
all_ngs

,ngram,Title,0
0,"(-2, -2, -2, -2)",Ave Maria,34
1,"(-2, -2, -2, -2)",Missa Confitemini: Gloria,10
2,"(-2, -2, -2, -3)",Ave Maria,4
3,"(-2, -2, -2, 1)",Ave Maria,8
4,"(-2, -2, -2, 1)",Missa Confitemini: Gloria,1
...,...,...,...
546,"(8, -3, 1, 4)",Ave Maria,1
547,"(8, -4, 2, -2)",Ave Maria,1
548,"(8, 1, 1, -5)",Ave Maria,1
549,"(8, 1, 1, 1)",Missa Confitemini: Gloria,1


In [80]:
all_ngs.groupby(by=["ngram", "Title"]).size()

ngram             Title                    
(-2, -2, -2, -2)  Ave Maria                    1
                  Missa Confitemini: Gloria    1
(-2, -2, -2, -3)  Ave Maria                    1
(-2, -2, -2, 1)   Ave Maria                    1
                  Missa Confitemini: Gloria    1
                                              ..
(8, -3, 1, 4)     Ave Maria                    1
(8, -4, 2, -2)    Ave Maria                    1
(8, 1, 1, -5)     Ave Maria                    1
(8, 1, 1, 1)      Missa Confitemini: Gloria    1
(8, 2, -3, 2)     Ave Maria                    1
Length: 551, dtype: int64

In [75]:
grouped_ngs = all_ngs.groupby(by=["ngram", "Title"])['Title'].size().reset_index(name='counts')


grouped_ngs#

,ngram,Title,counts
0,"(-2, -2, -2, -2)",Ave Maria,1
1,"(-2, -2, -2, -2)",Missa Confitemini: Gloria,1
2,"(-2, -2, -2, -3)",Ave Maria,1
3,"(-2, -2, -2, 1)",Ave Maria,1
4,"(-2, -2, -2, 1)",Missa Confitemini: Gloria,1
...,...,...,...
546,"(8, -3, 1, 4)",Ave Maria,1
547,"(8, -4, 2, -2)",Ave Maria,1
548,"(8, 1, 1, -5)",Ave Maria,1
549,"(8, 1, 1, 1)",Missa Confitemini: Gloria,1


In [64]:

grouped_ngs = all_ngs.groupby(by=["ngram", "Title"]).count().unstack(fill_value=0)
# .size().unstack(fill_value=0)
grouped_ngs.head(50)

counts                         
Title            Missa Confitemini: Gloria Missa Confitemini: Kyrie
ngram                                                              
(-2, -2, -2, -2)                         1                        1
(-2, -2, -2, 1)                          1                        1
(-2, -2, -2, 2)                          1                        1
(-2, -2, -2, 3)                          1                        0
(-2, -2, -2, 4)                          0                        1
(-2, -2, -3, 2)                          1                        0
(-2, -2, 1, -2)                          1                        1
(-2, -2, 1, 2)                           1                        1
(-2, -2, 2, -2)                          1                        1
(-2, -2, 2, -3)                          1                        1
(-2, -2, 2, 1)                           1                        1
(-2, -2, 2, 2)                           1                        1
(-2, -2, 2, 4)                           1                        1
(-2, -2, 3, 1)                           1                        0
(-2, -2, 4, 1)                           0                        1
(-2, -3, 2, -2)                          0                        1
(-2, -3, 2, -3)                          1                        0
(-2, -3, 2, 1)                           1                        1
(-2, -3, 2, 2)                           1                        1
(-2, -3, 4, -2)                          0                        1
(-2, -3, 4, 2)                           1                        0
(-2, -3, 5, 2)                           1                        0
(-2, -5, 4, -2)                          1                        1
(-2, -5, 4, -4)                          0                        1
(-2, -5, 4, 1)                           1                        0
(-2, -5, 5, 2)                           1                        0
(-2, 1, -2, -2)                          1                        1
(-2, 1, -2, 1)                           1                        0
(-2, 1, -2, 2)                           1                        0
(-2, 1, -5, 2)                           1                        0
(-2, 1, -5, 8)                           1                        0
(-2, 1, 1, -3)                           1                        0
(-2, 1, 1, 4)                            0                        1
(-2, 1, 2, -3)                           1                        0
(-2, 1, 2, 2)                            1                        1
(-2, 2, -2, -2)                          1                        1
(-2, 2, -2, 1)                           1                        0
(-2, 2, -2, 2)                           1                        1
(-2, 2, -2, 4)                           1                        1
(-2, 2, -2, 5)                           1                        0
(-2, 2, -3, -2)                          1                        0
(-2, 2, -3, 1)                           1                        0
(-2, 2, -3, 2)                           1                        1
(-2, 2, -3, 4)                           1                        1
(-2, 2, -4, 1)                           0                        1
(-2, 2, -4, 2)                           1                        1
(-2, 2, -4, 4)                           1                        0
(-2, 2, -8, 1)                           0                        1
(-2, 2, 1, -2)                           1                        1
(-2, 2, 1, -3)                           1                        0

## Corpus Search for Melodic nGrams

Note that the 'mel_corpus' much match the name of the combined results created above

In [9]:

def _convertTuple(tup):
    out = ""
    if isinstance(tup, tuple):
        out = ', '.join(tup)
    return out

@interact
def mel_ngram_search(my_search="", df = fixed(mel_corpus)):
    df_no_tuple = df.applymap(_convertTuple)
    df_no_tuple.pop("Composer")
    df_no_tuple.pop("Title")
    df_no_tuple.insert(0, "Composer", df["Composer"])
    df_no_tuple.insert(1, "Title", df["Title"])
    filtered_ngrams = df_no_tuple[df_no_tuple.apply(lambda x: x.astype(str).str.contains(my_search).any(), axis=1)].copy()
    
    pd.set_option('max_columns', None)
    return filtered_ngrams.fillna("-").reset_index().applymap(str).style.applymap(lambda x: "background: #ccebc4" if re.search(my_search, x) else "")

interactive(children=(Text(value='', description='my_search'), Output()), _dom_classes=('widget-interact',))

## Corpus Harmonic nGrams


* Set the **kind** ("d" = diatonic, "c" = chromatic) via **kwargs** below.
* Set the length (**n**) of ngrams via **kwargs** below.


In [10]:
def _convertTuple(tup):
    out = ""
    if isinstance(tup, tuple):
        out = ', '.join(tup)
    return out

func1 = ImportedPiece.harmonic
list_of_dfs = corpus.batch(func=func1, kwargs={'kind': 'd'}, metadata=True)
func2 = ImportedPiece.ngrams
list_of_harmonic_ngrams = corpus.batch(func=func2, kwargs={'n': 4, 'df': list_of_dfs})
func3 = ImportedPiece.detailIndex
list_of_detail_index = corpus.batch(func=func3, kwargs={'offset':False,'df': list_of_harmonic_ngrams}, metadata=True)
cleaned_list = []
for df in list_of_detail_index:
    df_no_tuple = df.applymap(_convertTuple)
    df_no_tuple["Composer"] = df["Composer"]
    df_no_tuple["Title"] = df["Title"] 
    cleaned_list.append(df_no_tuple)
har_corpus = pd.concat(cleaned_list)
har_corpus.sort_index(axis=1, inplace=True, ascending=False)
c = har_corpus.pop("Composer")
t = har_corpus.pop("Title")
har_corpus.insert(0, "Composer", c)
har_corpus.insert(1, "Title", t)

har_corpus

Composer                      Title         4_3  \
Measure Beat                                                        
1       1.0   Pierre Colin   Missa Confitemini: Kyrie  3, 3, 5, 3   
        4.0   Pierre Colin   Missa Confitemini: Kyrie  3, 5, 3, 5   
2       1.0   Pierre Colin   Missa Confitemini: Kyrie  5, 3, 5, 4   
        3.0   Pierre Colin   Missa Confitemini: Kyrie  3, 5, 4, 3   
3       1.0   Pierre Colin   Missa Confitemini: Kyrie  5, 4, 3, 6   
...                    ...                        ...         ...   
78      3.0   Pierre Colin  Missa Confitemini: Gloria  5, 8, 6, 4   
        4.0   Pierre Colin  Missa Confitemini: Gloria  8, 6, 4, 3   
79      1.0   Pierre Colin  Missa Confitemini: Gloria  6, 4, 3, 5   
        2.0   Pierre Colin  Missa Confitemini: Gloria  4, 3, 5, 1   
        2.5   Pierre Colin  Missa Confitemini: Gloria               

                      4_2             4_1         3_2         3_1         2_1  
Measure Beat                                                                   
1       1.0    5, 5, 8, 5     8, 8, 10, 8  3, 3, 4, 3  6, 6, 6, 6  4, 4, 3, 4  
        4.0    5, 8, 5, 8    8, 10, 8, 10  3, 4, 3, 4  6, 6, 6, 6  4, 3, 4, 3  
2       1.0    8, 5, 8, 7    10, 8, 10, 9  4, 3, 4, 4  6, 6, 6, 6  3, 4, 3, 3  
        3.0    5, 8, 7, 6    8, 10, 9, 10  3, 4, 4, 3  6, 6, 6, 8  4, 3, 3, 4  
3       1.0    8, 7, 6, 5    10, 9, 10, 8  4, 4, 3, 3  6, 6, 8, 6  3, 3, 4, 6  
...                   ...             ...         ...         ...         ...  
78      3.0   5, 10, 8, 6  10, 13, 12, 10  1, 3, 3, 3  6, 6, 7, 7  6, 4, 5, 5  
        4.0   10, 8, 6, 8  13, 12, 10, 11  3, 3, 3, 4  6, 7, 7, 8  4, 5, 5, 4  
79      1.0    8, 6, 8, 5  12, 10, 11, 10  3, 3, 4, 4  7, 7, 8, 7  5, 5, 4, 3  
        2.0                 10, 11, 10, 8  3, 4, 4, 5  7, 8, 7, 6  5, 4, 3, 4  
        2.5                                            8, 7, 6, 8              

[485 rows x 8 columns]

## Corpus Search for Harmonic nGrams

* Note that the length of ngrams is set above via the code for **har_corpus**

In [11]:
def _convertTuple(tup):
    out = ""
    if isinstance(tup, tuple):
        out = ', '.join(tup)
    return out

@interact
def har_ngram_search(my_search="", df = fixed(har_corpus)):
    df2 = har_corpus.copy()
    filtered_ngrams = df2[df2.apply(lambda x: x.astype(str).str.contains(my_search).any(), axis=1)].copy()
    
    pd.set_option('max_columns', None)
    return filtered_ngrams.fillna("-").reset_index().applymap(str).style.applymap(lambda x: "background: #ccebc4" if re.search(my_search, x) else "")

interactive(children=(Text(value='', description='my_search'), Output()), _dom_classes=('widget-interact',))